In [1]:
import time

In [2]:
# Config
file = "data/test-graph.txt"
max_iter = 100
num_partition = 4
debug = False

In [3]:
# Load the data
textFile = sc.textFile(file)
print("All lines:", textFile.count())
dataFile = textFile.filter(lambda l: len(l) > 0 and l[0] != '#')
print("Correct lines:", dataFile.count())

All lines: 16
Correct lines: 11


In [4]:
# Create edges RDD
def to_int_tuple(line, delim='\t'):
    strings = line.split(delim)[:2]
    return (int(strings[0]), int(strings[1]))
    
edges = dataFile.map(to_int_tuple)
edges.partitionBy(num_partition)
edges.cache()

# Check that all rows have exactly 2 entries
assert edges.filter(lambda t: len(t) == 2).count() == dataFile.count()

In [5]:
# Helper functions
def switch_key_value(kv):
    " Switch key with value. "
    return (kv[1], kv[0])

def compose(r1, r2):
    """ Compose 2 relations represented by PairRDDs. """
    r1_flip = r1.map(switch_key_value)
    # The key is now the intermediate node
    joined = r1_flip.join(r2, num_partition)
    return joined.values()

In [6]:
print("############# RDD: method 1 (single steps) ###############")
new_paths = edges
all_paths = edges

start = time.time()
true_start = start

# invariant:
###  - all_paths and new_paths are on 'num_partitions' partitions

for i in range(1, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    new_paths = compose(new_paths, edges)
    # Leave only really new paths
    new_paths = new_paths.subtract(all_paths).distinct(num_partition)
    new_paths.cache()
    print("Number of new paths: %d\n" % (new_paths.count(),))
    
    # Add new paths to all paths
    all_paths = all_paths.union(new_paths).coalesce(num_partition)
    all_paths.cache()
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
    
    # Finish, when no more paths added
    if new_paths.isEmpty():
        print("No new paths, finishing...")
        break

print("\n\n________________________________")
print("Total paths found: %d" % (all_paths.count(),))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method1_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method1_time,))
print("________________________________\n\n")

############# RDD: method 1 (single steps) ###############
________________________________
Iteration #1:
Number of new paths: 13

[(5, 9), (10, 8), (2, 4), (4, 6), (0, 2), (10, 7), (9, 6), (6, 10), (1, 3), (3, 5), (5, 7), (10, 9), (5, 8)] 

Iteration time: 1.484113 s.
________________________________
Iteration #2:
Number of new paths: 13

[(9, 9), (4, 9), (5, 10), (4, 8), (9, 7), (10, 10), (6, 6), (4, 7), (1, 4), (3, 6), (2, 5), (9, 8), (0, 3)] 

Iteration time: 1.888705 s.
________________________________
Iteration #3:
Number of new paths: 7

[(1, 5), (3, 7), (4, 10), (3, 8), (2, 6), (3, 9), (0, 4)] 

Iteration time: 1.675526 s.
________________________________
Iteration #4:
Number of new paths: 6

[(2, 8), (2, 7), (1, 6), (0, 5), (3, 10), (2, 9)] 

Iteration time: 1.308889 s.
________________________________
Iteration #5:
Number of new paths: 5

[(0, 6), (1, 9), (1, 7), (2, 10), (1, 8)] 

Iteration time: 1.349782 s.
________________________________
Iteration #6:
Number of new paths:

In [7]:
print("############# RDD: method 2 (paths combining) ###############")
new_paths = edges
all_paths = edges

start = time.time()
true_start = start

# invariant:
###  - all_paths and new_paths are on 'num_partitions' partitions

for i in range(1, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    new_paths = compose(all_paths, all_paths)
    # Leave only really new paths
    new_paths = new_paths.subtract(all_paths).distinct(num_partition)
    new_paths.cache()
    print("Number of new paths: %d\n" % (new_paths.count(),))
    
    # Add new paths to all paths
    all_paths = all_paths.union(new_paths).coalesce(num_partition)
    all_paths.cache()
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
        
    # Finish, when no more paths added
    if new_paths.isEmpty():
        print("No new paths, finishing...")
        break
        

print("\n\n________________________________")
print("Total paths found: %d" % (all_paths.count(),))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method2_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method2_time,))
print("________________________________\n\n")

############# RDD: method 2 (paths combining) ###############
________________________________
Iteration #1:
Number of new paths: 13

[(5, 9), (10, 8), (2, 4), (4, 6), (0, 2), (10, 7), (9, 6), (6, 10), (1, 3), (3, 5), (5, 7), (10, 9), (5, 8)] 

Iteration time: 0.900975 s.
________________________________
Iteration #2:
Number of new paths: 20

[(3, 7), (9, 9), (1, 5), (4, 10), (3, 8), (5, 10), (4, 9), (2, 6), (6, 6), (0, 4), (3, 9), (4, 8), (10, 10), (9, 7), (4, 7), (1, 4), (3, 6), (2, 5), (9, 8), (0, 3)] 

Iteration time: 1.173899 s.
________________________________
Iteration #3:
Number of new paths: 16

[(0, 6), (0, 10), (1, 9), (2, 8), (2, 7), (0, 9), (1, 10), (0, 5), (1, 6), (0, 8), (2, 10), (1, 7), (1, 8), (3, 10), (0, 7), (2, 9)] 

Iteration time: 1.442550 s.
________________________________
Iteration #4:
Number of new paths: 0

No new paths, finishing...


________________________________
Total paths found: 60
Number of iterations: #4

[(0, 1), (4, 5), (4, 6), (0, 2), (4, 10), (4

In [8]:
print("############# RDD: method 3 (paths combining 'optimized') ###############")

start = time.time()
true_start = start

all_paths = edges
new_paths = compose(edges, edges)
new_paths = new_paths.subtract(all_paths).distinct(num_partition)
# invariant:
###  - all_paths and new_paths are disjoint
###  - all_paths and new_paths are on 'num_partitions' partitions

for i in range(2, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    # Obtain new paths by composing old ones
    all_x_new_paths = compose(all_paths, new_paths)
    new_x_all_paths = compose(new_paths, all_paths)
    new_x_new_paths = compose(new_paths, new_paths)
    # Leave only really new paths
    all_paths = all_paths.union(new_paths).coalesce(num_partition)
    all_paths.cache()
    new_paths = all_x_new_paths.union(new_x_all_paths).union(new_x_new_paths)
    new_paths = new_paths.subtract(all_paths).distinct(num_partition)
    new_paths.cache()
    print("Number of new paths: %d\n" % (new_paths.count(),))
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
    
    # Finish, when no more paths added
    if new_paths.isEmpty():
        print("No new paths, finishing...")
        break
        

print("\n\n________________________________")
print("Total paths found: %d" % (all_paths.count(),))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method3_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method3_time,))
print("________________________________\n\n")

############# RDD: method 3 (paths combining 'optimized') ###############
________________________________
Iteration #2:
Number of new paths: 20

[(3, 7), (9, 9), (1, 5), (4, 10), (3, 8), (5, 10), (4, 9), (4, 8), (2, 6), (0, 4), (3, 9), (6, 6), (9, 7), (10, 10), (4, 7), (1, 4), (3, 6), (2, 5), (9, 8), (0, 3)] 

Iteration time: 2.943943 s.
________________________________
Iteration #3:
Number of new paths: 16

[(2, 8), (0, 6), (0, 10), (1, 9), (1, 10), (0, 9), (1, 6), (0, 5), (2, 7), (0, 8), (2, 10), (1, 7), (3, 10), (0, 7), (1, 8), (2, 9)] 

Iteration time: 2.569460 s.
________________________________
Iteration #4:
Number of new paths: 0

No new paths, finishing...


________________________________
Total paths found: 60
Number of iterations: #4

[(0, 1), (4, 5), (4, 6), (0, 2), (4, 10), (4, 9), (4, 8), (0, 4), (4, 7), (0, 3), (0, 6), (0, 10), (0, 9), (0, 5), (0, 8), (0, 7), (1, 2), (5, 6), (9, 10), (5, 9), (9, 6), (1, 3), (5, 7), (5, 8), (9, 9), (1, 5), (5, 10), (9, 7), (1, 4), (9, 8)

In [10]:
print("########### Summary ############")
print("Method 1: %f s." % (method1_time,))
print("Method 2: %f s." % (method2_time,))
print("Method 3: %f s." % (method3_time,))

########### Summary ############
Method 1: 12.259199 s.
Method 2: 5.146566 s.
Method 3: 7.822109 s.
